In [23]:
import chromadb
client = chromadb.PersistentClient('chroma/')
# chroma_client = chromadb.HttpClient(host="localhost", port=8000)
print(client.list_collections())

[Collection(name=ten_files_openai_3072), Collection(name=ten_files_openai_3072_tmp)]


In [25]:
import chromadb
client = chromadb.PersistentClient('../../../chroma/')
# chroma_client = chromadb.HttpClient(host="localhost", port=8000)
print(client.list_collections())

[Collection(name=ten_files_child), Collection(name=test), Collection(name=address_pdf_multivector_summary_240611), Collection(name=ten_files_openai_3072_child), Collection(name=ten_files), Collection(name=ten_files_openai_3072), Collection(name=address_ordinance_pdf), Collection(name=address_business_guide_pdf), Collection(name=address_regulation_pdf), Collection(name=address_law_pdf), Collection(name=address_law_txt), Collection(name=address_pdf_multivector_240611), Collection(name=address_pdf_vanila_240611), Collection(name=ten_files_openai_3072_tmp), Collection(name=ten_files_openai)]


In [26]:
from dotenv import load_dotenv
import os
import uuid
import chromadb
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import TextLoader
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever
from langchain_core.documents import Document
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain_community.embeddings import OllamaEmbeddings
from langchain.embeddings.openai import OpenAIEmbeddings
from chromadb.utils import embedding_functions
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI
from kiwipiepy import Kiwi
import gradio as gr
import time
import csv


# .env 파일 활성화
load_dotenv()

from kiwipiepy import Kiwi

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

############################################################################################################
######### 파일 불러오기 #######################################################################################
############################################################################################################

loader = DirectoryLoader(path='data/', glob='*.txt', loader_cls=TextLoader, show_progress=True, use_multithreading=True)
data = loader.load()

############################################################################################################
######### 컬렉션 생성/연결하기 ##################################################################################
############################################################################################################
client = chromadb.PersistentClient('../../chroma/')
collection = client.get_or_create_collection(name="ten_files_openai_3072")

############################################################################################################
############# txt doc 설정하기 ################################################################################
############################################################################################################
doc_ids = [str(uuid.uuid4()) for _ in data]
child_text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 200)

id_key = "doc_id"

child_docs = []  # 하위 문서를 저장할 리스트를 초기화합니다.
for i, doc in enumerate(data):
    _id = doc_ids[i]  # 현재 문서의 ID를 가져옵니다.
    # 현재 문서를 하위 문서로 분할합니다.
    child_doc = child_text_splitter.split_documents([doc])
    for _doc in child_doc:  # 분할된 하위 문서에 대해 반복합니다.
        # 하위 문서의 메타데이터에 ID를 저장합니다.
        _doc.metadata[id_key] = _id
    child_docs.extend(child_doc)  # 분할된 하위 문서를 리스트에 추가합니다.

100%|██████████| 10/10 [00:00<00:00, 1294.50it/s]


In [11]:
child_docs

[Document(page_content='도로명주소법\n[시행 2021. 6. 9.] [법률 제17574호, 2020. 12. 8., 전부개정]\n행정안전부 (주소생활공간과) 044-205-3567\n \n제1조(목적) 이 법은 도로명주소, 국가기초구역, 국가지점번호 및 사물주소의 표기ㆍ사용ㆍ관리ㆍ활용 등에 관한 사항\n을 규정함으로써 국민의 생활안전과 편의를 도모하고 관련 산업의 지원을 통하여 국가경쟁력 강화에 이바지함을 목\n적으로 한다.\n \n제2조(정의) 이 법에서 사용하는 용어의 뜻은 다음과 같다.\n1. “도로”란 다음 각 목의 어느 하나에 해당하는 것을 말한다.\n가. 「도로법」 제2조제1호에 따른 도로(같은 조 제2호에 따른 도로의 부속물은 제외한다)\n나. 그 밖에 차량 등 이동수단이나 사람이 통행할 수 있는 통로로서 대통령령으로 정하는 것\n2. “도로구간”이란 도로명을 부여하기 위하여 설정하는 도로의 시작지점과 끝지점 사이를 말한다.\n3. “도로명”이란 도로구간마다 부여된 이름을 말한다.', metadata={'source': 'data/address_law.txt', 'doc_id': 'd7634712-54d2-418f-8d27-0327a739bc56'}),
 Document(page_content='가. 「도로법」 제2조제1호에 따른 도로(같은 조 제2호에 따른 도로의 부속물은 제외한다)\n나. 그 밖에 차량 등 이동수단이나 사람이 통행할 수 있는 통로로서 대통령령으로 정하는 것\n2. “도로구간”이란 도로명을 부여하기 위하여 설정하는 도로의 시작지점과 끝지점 사이를 말한다.\n3. “도로명”이란 도로구간마다 부여된 이름을 말한다.\n4. “기초번호”란 도로구간에 행정안전부령으로 정하는 간격마다 부여된 번호를 말한다.\n5. “건물번호”란 다음 각 목의 어느 하나에 해당하는 건축물 또는 구조물(이하 “건물등”이라 한다)마다 부여된 번호\n(둘 이상의 건물등이 하나의 집단을 형성하고 있는 경우로서 대통령령으로 정하는 경우에는 그 건물

In [27]:
# BM25 retrieval 
bm25_retriever = BM25Retriever.from_documents(child_docs) #preprocess_func = kiwi_tokenize
bm25_retriever.k = 1  # BM25Retriever의 검색 결과 개수를 1로 설정합니다.

#DB retrieval 
embedding = OpenAIEmbeddings(model='text-embedding-3-large')  # OpenAI 임베딩을 사용합니다.

vectorstore = Chroma(client=client, collection_name="ten_files_openai_3072", embedding_function=embedding)
chromadb_retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 1})

# Ensemble retrieval 
ensemble_retriever = EnsembleRetriever(
    retrievers=[chromadb_retriever],
    weights=[1],
    search_type="mmr",
)


In [28]:
query = '건물번호 부여 방법에 대해서 알려줘'
ensemble_result = ensemble_retriever.invoke(query)
ensemble_result

[]